In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline
plt.rcParams['figure.figsize'] = (16,10)

In [2]:
def read_csv(TRAIN_PATH):
    chunksize = 5_000_000 # batch size
    
    traintypes = {
        'fare': 'float32',
        'from': 'str',
        'to': 'str',
        'p_count': 'uint8', 
        'hour': 'uint8',
        'weekday': 'uint8',
        'week': 'uint8',
        'year': 'int16',
        'dist_e': 'float32',
        'dist_t': 'float32',
        'x0': 'float32',
        'x1': 'float32',
        'y0': 'float32',
        'y1': 'float32',
    }
    
    cols = list(traintypes.keys())
    
    # loading the dataframe into list of small dataframes
    df_list = []

    for df_chunk in pd.read_csv(TRAIN_PATH, usecols=cols, dtype=traintypes, chunksize=chunksize):
        df_list.append(df_chunk) 
        df = pd.concat(df_list)
        
    return df

In [3]:
train = read_csv('../input/train_cleaned_2.csv')

In [4]:
train.head()

,fare,from,to,hour,weekday,week,year,p_count,dist_e,dist_t,x0,x1,y0,y1
0,4.5,Queens,Queens,17,0,25,2009,1,0.638322,0.759079,-73.844315,-73.841614,40.721317,40.712276
1,16.9,Manhattan,Manhattan,16,1,1,2010,1,5.213395,6.717119,-74.016045,-73.979271,40.711304,40.782005
2,5.7,Manhattan,Manhattan,0,3,33,2011,2,0.852610,1.164240,-73.982735,-73.991241,40.761269,40.750561
3,7.7,Manhattan,Manhattan,4,5,16,2012,1,1.735600,1.943343,-73.987129,-73.991570,40.733143,40.758091
4,5.3,Manhattan,Manhattan,7,1,10,2010,1,1.228196,1.658726,-73.968094,-73.956657,40.768009,40.783764


In [5]:
y = train['fare']
X = train.drop(['fare'], axis=1)
del train

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.23, random_state=379582)

del X,y

In [12]:
locations = [
    'Manhattan', 'JFK', 'LaGardia', 'Newark', 'Teterboro', 'Brooklin', 'Bronx', 'Staten', 'Queens', 'Westchester',
    'Nassau', 'Jersey', 'Bayonne', 'WStaton', 'West', 'WNewYork', 'North'        
]

# num_buckets = [0, num_buckets)

p_count = tf.feature_column.numeric_column('p_count')
taxicab = tf.feature_column.numeric_column('dist_t')
euclidean = tf.feature_column.numeric_column('dist_e')

x0 = tf.feature_column.numeric_column('x0')
x1 = tf.feature_column.numeric_column('x1')
y0 = tf.feature_column.numeric_column('y0')
y1 = tf.feature_column.numeric_column('y1')

numbuckets = 35
dx = np.linspace(-74.27, -72.98, numbuckets).tolist()
dy = np.linspace(40.56, 41.71, numbuckets).tolist()

x0_B = tf.feature_column.bucketized_column(x0, dx)
x1_B = tf.feature_column.bucketized_column(x1, dx)
y0_B = tf.feature_column.bucketized_column(y0, dy)
y1_B = tf.feature_column.bucketized_column(y1, dy)

xy_0 = tf.feature_column.crossed_column([x0_B, y0_B], numbuckets * numbuckets)
xy_1 = tf.feature_column.crossed_column([x1_B, y1_B], numbuckets * numbuckets)
xy_pair = tf.feature_column.crossed_column([xy_0, xy_1], numbuckets**4)

p_count_b = tf.feature_column.bucketized_column(p_count, [1,2,3])

# definition of the categorical columns
hour = tf.feature_column.categorical_column_with_identity('hour', num_buckets = 24)
weekday = tf.feature_column.categorical_column_with_identity('weekday', num_buckets = 7)
week = tf.feature_column.categorical_column_with_identity('week', num_buckets = 54)
year = tf.feature_column.categorical_column_with_vocabulary_list('year', [2009, 2010, 2011, 2012, 2013, 2014, 2015])

loc_from = tf.feature_column.categorical_column_with_vocabulary_list('from', locations)
loc_to = tf.feature_column.categorical_column_with_vocabulary_list('to', locations)

# definition of the feature crosses
from_to = tf.feature_column.crossed_column([loc_from, loc_to], len(locations)**2)
from_to_year = tf.feature_column.crossed_column([from_to, year], len(locations)**2 * 7)
from_to_year_week = tf.feature_column.crossed_column([from_to_year, week], len(locations)**2 * 7 * 54)
from_to_p_count = tf.feature_column.crossed_column([from_to, p_count_b], len(locations)**2 * 5)

hour_weekday = tf.feature_column.crossed_column([hour, weekday], 24 * 7)
weekday_week_year = tf.feature_column.crossed_column([weekday, week, year], 7 * 54 * 7)
week_year = tf.feature_column.crossed_column([week, year], 54 * 7)
weekday_year = tf.feature_column.crossed_column([weekday, year], 7 * 7)
hour_weekday_year = tf.feature_column.crossed_column([hour, weekday, year], 24 * 7 * 7)

In [13]:
deep_columns = [
    # Embedding_column to "group" together ...
    tf.feature_column.embedding_column(from_to, 50),
    tf.feature_column.embedding_column(from_to_year, 50),
    tf.feature_column.embedding_column(from_to_year_week, 50),
    tf.feature_column.embedding_column(hour_weekday, 50),
    tf.feature_column.embedding_column(weekday_week_year, 50),
    tf.feature_column.embedding_column(week_year,50),
    tf.feature_column.embedding_column(weekday_year,50),
    tf.feature_column.embedding_column(hour_weekday_year, 50),
    tf.feature_column.embedding_column(xy_pair, 50),
    tf.feature_column.embedding_column(from_to_p_count, 50),
    

    # indicator columns
    tf.feature_column.indicator_column(loc_from),
    tf.feature_column.indicator_column(loc_to),
    tf.feature_column.indicator_column(hour),
    tf.feature_column.indicator_column(weekday),
    tf.feature_column.indicator_column(week),
    tf.feature_column.indicator_column(year),
    tf.feature_column.indicator_column(xy_0),
    tf.feature_column.indicator_column(xy_1),
    tf.feature_column.indicator_column(p_count_b),

    # numeric columns
    taxicab, euclidean, p_count
]

In [14]:
def train_input_fn(features, labels, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        num_epochs = 1,
        shuffle = False)

In [15]:
model_dir = './DNN_Regressor_regions_coords_overfit_crosses'

file_writer = tf.summary.FileWriter(model_dir)

estimator = tf.estimator.DNNRegressor(
    model_dir = model_dir,
    feature_columns = deep_columns,
    optimizer = tf.train.AdamOptimizer(learning_rate=0.00001),
    hidden_units = [128,128,128,128], 
    batch_norm = True,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './DNN_Regressor_regions_coords_overfit_crosses', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000211AB762630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(X_train, y_train, batch_size=3000))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(X_eval, y_eval))

In [17]:
for i in range(20):
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./DNN_Regressor_regions_coords_overfit_crosses\model.ckpt.
INFO:tensorflow:loss = 584619.1, step = 0
INFO:tensorflow:global_step/sec: 4.62338
INFO:tensorflow:loss = 538407.25, step = 100 (21.637 sec)
INFO:tensorflow:global_step/sec: 4.96216
INFO:tensorflow:loss = 501492.34, step = 200 (20.146 sec)
INFO:tensorflow:global_step/sec: 5.08211
INFO:tensorflow:loss = 533347.6, step = 300 (19.678 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:global_step/sec: 5.22886
INFO:tensorflow:loss = 334751.22, step = 6300 (19.125 sec)
INFO:tensorflow:global_step/sec: 5.22442
INFO:tensorflow:loss = 312863.6, step = 6400 (19.141 sec)
INFO:tensorflow:global_step/sec: 5.22878
INFO:tensorflow:loss = 349469.38, step = 6500 (19.141 sec)
INFO:tensorflow:global_step/sec: 5.21162
INFO:tensorflow:loss = 313867.94, step = 6600 (19.172 sec)
INFO:tensorflow:global_step/sec: 5.21865
INFO:tensorflow:loss = 334437.75, step = 6700 (19.178 sec)
INFO:tensorflow:global_step/sec: 5.20269
INFO:tensorflow:loss = 324679.06, step = 6800 (19.205 sec)
INFO:tensorflow:global_step/sec: 5.22633
INFO:tensorflow:loss = 340306.7, step = 6900 (19.150 sec)
INFO:tensorflow:global_step/sec: 5.22631
INFO:tensorflow:loss = 303598.88, step = 7000 (19.134 sec)
INFO:tensorflow:global_step/sec: 5.24021
INFO:tensorflow:loss = 304837.3, step = 7100 (19.068 sec)
INFO:tensorflow:global_step/sec: 5.23081
INFO:tensorflow:loss = 293213.1, step = 7200 (19.133 sec)
INFO

INFO:tensorflow:loss = 191171.3, step = 13100 (19.142 sec)
INFO:tensorflow:global_step/sec: 5.24366
INFO:tensorflow:loss = 190707.28, step = 13200 (19.086 sec)
INFO:tensorflow:global_step/sec: 5.22443
INFO:tensorflow:loss = 189063.67, step = 13300 (19.141 sec)
INFO:tensorflow:global_step/sec: 5.23069
INFO:tensorflow:loss = 193522.83, step = 13400 (19.102 sec)
INFO:tensorflow:Saving checkpoints for 13423 into ./DNN_Regressor_regions_coords_overfit_crosses\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-05-03:53:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions_coords_overfit_crosses\model.ckpt-13423
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation 

INFO:tensorflow:global_step/sec: 5.21381
INFO:tensorflow:loss = 101289.016, step = 18123 (19.180 sec)
INFO:tensorflow:global_step/sec: 5.2172
INFO:tensorflow:loss = 119519.84, step = 18223 (19.183 sec)
INFO:tensorflow:global_step/sec: 5.19606
INFO:tensorflow:loss = 96775.375, step = 18323 (19.230 sec)
INFO:tensorflow:global_step/sec: 5.20267
INFO:tensorflow:loss = 114322.21, step = 18423 (19.221 sec)
INFO:tensorflow:global_step/sec: 5.22588
INFO:tensorflow:loss = 82478.51, step = 18523 (19.136 sec)
INFO:tensorflow:global_step/sec: 5.2068
INFO:tensorflow:loss = 106443.414, step = 18623 (19.206 sec)
INFO:tensorflow:global_step/sec: 5.22732
INFO:tensorflow:loss = 91427.11, step = 18723 (19.130 sec)
INFO:tensorflow:global_step/sec: 5.21356
INFO:tensorflow:loss = 107860.2, step = 18823 (19.181 sec)
INFO:tensorflow:global_step/sec: 5.23795
INFO:tensorflow:loss = 85374.82, step = 18923 (19.107 sec)
INFO:tensorflow:global_step/sec: 5.21437
INFO:tensorflow:loss = 87236.234, step = 19023 (19.162

INFO:tensorflow:global_step/sec: 5.22658
INFO:tensorflow:loss = 38715.348, step = 24923 (19.133 sec)
INFO:tensorflow:global_step/sec: 5.22236
INFO:tensorflow:loss = 30921.775, step = 25023 (19.148 sec)
INFO:tensorflow:global_step/sec: 5.22018
INFO:tensorflow:loss = 32450.531, step = 25123 (19.156 sec)
INFO:tensorflow:global_step/sec: 5.21599
INFO:tensorflow:loss = 34551.734, step = 25223 (19.172 sec)
INFO:tensorflow:global_step/sec: 5.22965
INFO:tensorflow:loss = 153800.52, step = 25323 (19.122 sec)
INFO:tensorflow:global_step/sec: 5.22781
INFO:tensorflow:loss = 32859.062, step = 25423 (19.144 sec)
INFO:tensorflow:global_step/sec: 5.23056
INFO:tensorflow:loss = 64178.59, step = 25523 (19.103 sec)
INFO:tensorflow:global_step/sec: 5.2313
INFO:tensorflow:loss = 39640.55, step = 25623 (19.116 sec)
INFO:tensorflow:global_step/sec: 5.22437
INFO:tensorflow:loss = 47818.625, step = 25723 (19.141 sec)
INFO:tensorflow:Saving checkpoints for 25799 into ./DNN_Regressor_regions_coords_overfit_cross

...

INFO:tensorflow:global_step/sec: 5.22892
INFO:tensorflow:loss = 29868.365, step = 36546 (19.140 sec)
INFO:tensorflow:global_step/sec: 5.20938
INFO:tensorflow:loss = 34988.16, step = 36646 (19.181 sec)
INFO:tensorflow:global_step/sec: 5.21424
INFO:tensorflow:loss = 25561.012, step = 36746 (19.178 sec)
INFO:tensorflow:global_step/sec: 5.22889
INFO:tensorflow:loss = 34455.465, step = 36846 (19.125 sec)
INFO:tensorflow:global_step/sec: 5.21858
INFO:tensorflow:loss = 51728.06, step = 36946 (19.162 sec)
INFO:tensorflow:global_step/sec: 5.2087
INFO:tensorflow:loss = 28238.93, step = 37046 (19.199 sec)
INFO:tensorflow:global_step/sec: 5.2145
INFO:tensorflow:loss = 38151.863, step = 37146 (19.179 sec)
INFO:tensorflow:global_step/sec: 5.23464
INFO:tensorflow:loss = 32285.129, step = 37246 (19.117 sec)
INFO:tensorflow:global_step/sec: 5.19923
INFO:tensorflow:loss = 30505.96, step = 37346 (19.218 sec)
INFO:tensorflow:global_step/sec: 5.22676
INFO:tensorflow:loss = 33576.027, step = 37446 (19.148 s

INFO:tensorflow:global_step/sec: 5.21823
INFO:tensorflow:loss = 27002.676, step = 41469 (19.164 sec)
INFO:tensorflow:global_step/sec: 5.21301
INFO:tensorflow:loss = 29720.16, step = 41569 (19.185 sec)
INFO:tensorflow:global_step/sec: 5.22345
INFO:tensorflow:loss = 34805.273, step = 41669 (19.142 sec)
INFO:tensorflow:global_step/sec: 5.21964
INFO:tensorflow:loss = 29435.875, step = 41769 (19.174 sec)
INFO:tensorflow:global_step/sec: 5.22888
INFO:tensorflow:loss = 28135.277, step = 41869 (19.125 sec)
INFO:tensorflow:global_step/sec: 5.21939
INFO:tensorflow:loss = 31219.594, step = 41969 (19.144 sec)
INFO:tensorflow:global_step/sec: 5.21895
INFO:tensorflow:loss = 25842.684, step = 42069 (19.161 sec)
INFO:tensorflow:global_step/sec: 5.22564
INFO:tensorflow:loss = 38158.633, step = 42169 (19.136 sec)
INFO:tensorflow:global_step/sec: 5.21704
INFO:tensorflow:loss = 36310.97, step = 42269 (19.168 sec)
INFO:tensorflow:global_step/sec: 5.20651
INFO:tensorflow:loss = 31538.018, step = 42369 (19.2

INFO:tensorflow:global_step/sec: 5.22718
INFO:tensorflow:loss = 41236.875, step = 47269 (19.131 sec)
INFO:tensorflow:global_step/sec: 5.17587
INFO:tensorflow:loss = 89481.94, step = 47369 (19.305 sec)
INFO:tensorflow:global_step/sec: 5.21716
INFO:tensorflow:loss = 35336.61, step = 47469 (19.168 sec)
INFO:tensorflow:global_step/sec: 5.20567
INFO:tensorflow:loss = 29911.137, step = 47569 (19.210 sec)
INFO:tensorflow:global_step/sec: 5.19898
INFO:tensorflow:loss = 31389.564, step = 47669 (19.235 sec)
INFO:tensorflow:global_step/sec: 5.21609
INFO:tensorflow:loss = 39245.07, step = 47769 (19.187 sec)
INFO:tensorflow:global_step/sec: 5.2116
INFO:tensorflow:loss = 31612.568, step = 47869 (19.172 sec)
INFO:tensorflow:global_step/sec: 5.22465
INFO:tensorflow:loss = 38638.742, step = 47969 (19.156 sec)
INFO:tensorflow:global_step/sec: 5.22009
INFO:tensorflow:loss = 26614.916, step = 48069 (19.157 sec)
INFO:tensorflow:global_step/sec: 5.22888
INFO:tensorflow:loss = 42628.19, step = 48169 (19.109 

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions_coords_overfit_crosses\model.ckpt-53692
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-09-05-06:04:48
INFO:tensorflow:Saving dict for global step 53692: average_loss = 11.231487, global_step = 53692, label/mean = 11.205854, loss = 1437.6304, prediction/mean = 11.097354
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 53692: ./DNN_Regressor_regions_coords_overfit_crosses\model.ckpt-53692
INFO:tensorflow:Loss for final step: 26539.543.
IN

ResourceExhaustedError: OOM when allocating tensor with shape[1500625,50] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: save/Assign_87 = Assign[T=DT_FLOAT, _class=["loc:@dnn/i...hts/part_0"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dnn/dnn/input_from_feature_columns/input_layer/x0_bucketized_X_x1_bucketized_X_y0_bucketized_X_y1_bucketized_embedding/embedding_weights/part_0/Adam_1, save/RestoreV2/_175)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'save/Assign_87', defined at:
  File "C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\vasil\AppData\Local\

In [18]:
infer = pd.read_csv('../input/test_processed.csv', index_col='key')

generator = estimator.predict(input_fn=pred_input_fn(infer), 
                              checkpoint_path='./DNN_Regressor_regions_coords_overfit_crosses/model.ckpt-46459')
predictions = [next(generator) for i in range(len(infer))]
values = [val['predictions'].tolist()[0] for val in predictions]

ids = infer.index
submission = pd.DataFrame()
submission['fare_amount'] = values
submission['key'] = ids
submission.set_index('key', inplace=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_Regressor_regions_coords_overfit_crosses/model.ckpt-46459
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [19]:
submission.head()

,fare_amount
key,
2015-01-27 13:08:24.0000002,11.659226
2015-01-27 13:08:24.0000003,10.694809
2011-10-08 11:53:44.0000002,4.634830
2012-12-01 21:12:12.0000002,8.870740
2012-12-01 21:12:12.0000003,15.955825


In [20]:
submission.to_csv('../output/23.dnn_regressor_rmse_3.26.csv')